## load Test Data


In [ ]:
from libs import *
from utils import *
dat = np.load('Test_PGA_instance.npz')
x_test = dat['x_test']
y_test = dat['y_test']

# Extract Max Value in Each Record.
mate = np.max(np.max(np.abs(x_test),axis=1),axis=-1)

## Load ViT Model

In [ ]:
def create_vit_classifier(inputs):
    
    filters = [64, 128, 96, 128, 256] 

    e = convF1(inputs, filters[1], 3, 0.1)
    
    e = convF1(e, filters[1], 3, 0.1)

    inputreshaped = layers.Reshape((400,1,128))(e)
    # Create patches.
    patches = Patches(patch_size)(inputreshaped)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    
    #print(patches, encoded_patches)
    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        #x3 = convF1(x3, projection_dim,11, 0.1)

        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)

    return representation

inp1 = layers.Input(shape=(400,3),name='input_layer')
inp2 = layers.Input(shape=(1,),name='input_layer1')

features = create_vit_classifier(inp1)
features = concatenate([features,inp2])

e = Dense(1)(features)
o = Activation('linear', name='output_layer')(e)
model = Model(inputs=[inp1,inp2], outputs=o)
model.summary()

model = load_model('best_model_RR3_3s_1sbefore_Mix100.h5', custom_objects={ "PatchEncoder":PatchEncoder,
                                                                                   "Patches":Patches,
                                                                                   "mlp":mlp})

## Predict Test Set

In [ ]:
outte = model.predict([x_test,mate],batch_size=1024,verbose=1)
outte = outte[:,0]

## Plotting the Prediction Vs Ground-Truth

In [ ]:
fig4, ax = plt.subplots()
ax.scatter(y_test, outte, alpha = 0.4, facecolors='none', edgecolors='r')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', alpha=0.4, lw=2)
ax.set_xlim([0,340])
ax.set_xlabel('PGA (Ground Truth)',fontsize='large', fontweight='bold')
ax.set_ylabel('PGA (Predicted)',fontsize='large', fontweight='bold')

diff = y_test - outte
diff = np.array(diff)
(np.round(np.mean(np.abs(diff)), 2)), (np.round(np.std(diff), 2))

## Plotting the Histogrma of the Prediction Error 

In [ ]:
plt.hist([diff],100, alpha=0.8,color='lightcoral', edgecolor='black',linewidth=1.2)
plt.xlim([-20,20])
plt.xlabel('PGA (cm/s/s)',fontsize='large', fontweight='bold')
plt.ylabel('Count',fontsize='large', fontweight='bold')
plt.text(
    0.8,
    0.7,
    'MAE = ' + str(np.round(np.mean(np.abs(diff)),3)),
    horizontalalignment='center',
    transform=plt.gca().transAxes,
    fontsize="medium",
    fontweight="bold")
plt.text(
    0.8,
    0.5,
    'STD = ' + str(np.round(np.std((diff)),3)),
    horizontalalignment='center',
    transform=plt.gca().transAxes,
    fontsize="medium",
    fontweight="bold")

